# Testing and running the SCF solver

In [ ]:
import sys
sys.path.append('/home/ngoel/programs/vampyr/build/lib/python3.13/site-packages')
import numpy as np
from vampyr import vampyr3d as vp
import system
import kain

precision = 1e-4
threshold = precision * 10

mra = system.initMRA(nBoxes=2, scaling = 1.0, scale = -4, order = 8)
# Water 1
atom1 = system.Atom(coords = (3.03640321, -0.125273197, 0.165537034), charge = 8)
atom2 = system.Atom(coords = (1.20775692, 0.126394035, 0.040864271), charge = 1)
atom3 = system.Atom(coords = (3.7449532, 1.653846757, 0.068992382), charge = 1)
# molecule = system.Molecule(atoms = [atom1, atom2, atom3], path="../../water_4/")

# Water 2
atom4 = system.Atom(coords = (-2.236881593, 0.079767042, -0.135775327), charge = 8)
atom5 = system.Atom(coords = (-2.80936204, -0.932460622, -1.444139354), charge = 1)
atom6 = system.Atom(coords = (-2.942869717, -0.602274003, 1.504521009), charge = 1)
# molecule = system.Molecule(atoms = [atom1, atom2, atom3, atom4, atom5, atom6], path="./../../diwater_5/")
he1 = system.Atom(coords = (0.0, 0.0, -1.5), charge = 2)
# he2 = Atom(coords = (0.0, 0.0, 1.5), charge = 2)
molecule = system.Molecule(atoms = [he1], path = "../../heA_4/")

scf = kain.SCF(molecule, mra, precision)
Phi, energies, updates = scf.runSCF(threshold = threshold, maxIter = 100, kain_start = 0, kain_history = 5)
print(updates)

: 

# Testing and Running the embedding implementation

In [ ]:
import sys
sys.path.append('/home/ngoel/programs/vampyr/build/lib/python3.13/site-packages')
import numpy as np
from vampyr import vampyr3d as vp
import system
import kain
import fde

precision = 1e-4
threshold = precision * 10

mra = system.initMRA(nBoxes=2, scaling = 1.0, scale = -4, order = 8)
# Water 1
# atom1 = system.Atom(coords = (3.03640321, -0.125273197, 0.165537034), charge = 8)
# atom2 = system.Atom(coords = (1.20775692, 0.126394035, 0.040864271), charge = 1)
# atom3 = system.Atom(coords = (3.7449532, 1.653846757, 0.068992382), charge = 1)
# molecule1 = system.Molecule(atoms = [atom1, atom2, atom3], path="../../water_4/")

# Water 2
# atom4 = system.Atom(coords = (-2.236881593, 0.079767042, -0.135775327), charge = 8)
# atom5 = system.Atom(coords = (-2.80936204, -0.932460622, -1.444139354), charge = 1)
# atom6 = system.Atom(coords = (-2.942869717, -0.602274003, 1.504521009), charge = 1)
# molecule2 = system.Molecule(atoms = [atom4, atom5, atom6], path="./../../diwater_5/")
he1 = system.Atom(coords = (0.0, 0.0, -1.5), charge = 2)
he2 = system.Atom(coords = (0.0, 0.0, 1.5), charge = 2)
moleculeA = system.Molecule(atoms = [he1], path = "../../heA_4/")
moleculeB = system.Molecule(atoms = [he2], path = "../../heB_4/")

scf = fde.FDE(moleculeA, moleculeB, mra, precision)
Phi, energies, updates = scf.runFDE(threshold = threshold, maxIter = 100, kain_start = 0, kain_history = 0)
print(updates)